In [1]:
import datacube
import time
import rasterio
from matplotlib import pyplot

dc = datacube.Datacube()

In [ ]:
dc.list_products()

In [ ]:
start = time.time()
accum_prcp = dc.load(product='accum_prcp')
#accum_prcp = dc.load(product='accum_prcp',id='1311c672-c2c5-5911-a2e5-2a76f487c143')
end = time.time()
print(end - start)



#accum_prcp2 = dc.load(product='accum_prcp',id='9bec1213-ed93-5cbc-9e39-893ec90961b8')
#accum_prcp = dc.load(product='accum_prcp', output_crs='epsg:3577', resolution=(-12500,12500))

In [ ]:
accum_prcp

In [ ]:
total_day_accumprcp = accum_prcp['accum_prcp'].sum(dim='time')

In [ ]:
pyplot.imshow(total_day_accumprcp, cmap='pink')

In [ ]:
accum_prcp2 = dc.load(product='accum_prcp',id='9bec1213-ed93-5cbc-9e39-893ec90961b8')

In [ ]:
accum_prcp2

In [ ]:
pyplot.imshow(accum_prcp2['accum_prcp'].squeeze(), cmap='pink')


In [ ]:
pyplot.imshow(accum_prcp['accum_prcp'].squeeze(), cmap='pink')


In [ ]:
total_day_accumprcp = accum_prcp['accum_prcp'].sum(dim='time')

In [ ]:
start = time.time()
src = rasterio.open('NetCDF:"/home/david/Downloads/BARRA/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19920302T0000Z.sub.nc":accum_prcp')
img=src.read(4)
end = time.time()
print(end - start)

In [ ]:
pyplot.imshow(img, cmap='pink')

In [ ]:
with rasterio.open('NetCDF:"/home/david/Downloads/BARRA/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19920302T0000Z.sub.nc":accum_prcp', 'r') as img:
    left, bottom, right, top = [i for i in img.bounds]
    spatial_reference = str(
        str(getattr(img, 'crs_wkt', None) or img.crs.wkt))
    res = img.res
    #return left, bottom, right, top, spatial_reference, res
    print(left)
    print(bottom)
    print(right)
    print(top)
    print(spatial_reference)
    print(res)

In [2]:
import dask
import dask.distributed

client = dask.distributed.Client(n_workers=2,
                                 threads_per_worker=1,
                                 processes=False, 
                                 ip='127.0.0.1')
client

Client Scheduler: tcp://127.0.0.1:40433 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 2.68 GB


In [3]:
%%time
xx = dc.load(product='accum_prcp',
             dask_chunks={'longitude': 2000, 'latitude': 2000})

print("Number of chunks per band: {}x{}x{}".format(*xx.accum_prcp.data.to_delayed().shape))
display(xx)

Number of chunks per band: 24x1x1


<xarray.Dataset>
Dimensions:     (latitude: 769, longitude: 1201, time: 24)
Coordinates:
  * time        (time) datetime64[ns] 1992-03-02T00:29:59.999999 ... 1992-03-02T23:29:59.999999
  * latitude    (latitude) float64 19.53 19.42 19.31 ... -64.74 -64.85 -64.96
  * longitude   (longitude) float64 64.96 65.07 65.18 ... 196.7 196.8 197.0
Data variables:
    accum_prcp  (time, latitude, longitude) float64 dask.array<shape=(24, 769, 1201), chunksize=(1, 769, 1201)>
Attributes:
    crs:      GEOGCS["unknown",DATUM["unknown",SPHEROID["Sphere",6371229,0]],...

CPU times: user 74.5 ms, sys: 16.5 ms, total: 91.1 ms
Wall time: 184 ms


In [4]:
%%time
total_day_accumprcp = xx['accum_prcp'].sum(dim='time')

CPU times: user 4.05 ms, sys: 3.28 ms, total: 7.34 ms
Wall time: 10.5 ms


In [ ]:
%%time
pyplot.imshow(total_day_accumprcp, cmap='pink')